# Setup

lsgraph was created with the aim of supporting reskilling and upskilling efforts for both individuals and organisations. lsgraph provides services to: 

* collect course details and associate with skills, 
* recommend courses for a learner, 
* recommend job profiles, 
* suggest reskilling options across a workforce.

This notebook walks through the process of setting up organisations, skills, profiles and users.

In [24]:
import json
import os
import pandas as pd
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1/"

# Create credentials

Access to the API is restricted by API keys that need to be included in the request headers.

If running lsgraph locally via the docker containers the below command should create a new customer and access credentials.

In [23]:
!python ../create-customer-account.py "Customer Name" newcustomer@test.local

/home/jonathan/miniconda3/lib/python3.6/site-packages/apispec/ext/marshmallow/common.py:131: UserWarning: Multiple schemas resolved to the name User. The name has been modified. Either manually add each of the schemas with a different name or provide a custom schema_name_resolver.
  UserWarning,
/home/jonathan/miniconda3/lib/python3.6/site-packages/apispec/ext/marshmallow/common.py:131: UserWarning: Multiple schemas resolved to the name Skill. The name has been modified. Either manually add each of the schemas with a different name or provide a custom schema_name_resolver.
  UserWarning,
Customer ID: 3a5c4f91-3ffd-4536-a0ad-8dd13bbc7e34
API Key: LSQHTIXqmeoFxntT31Up
Access Token: TwIJ92oO2PHeWfUe2IyBAhzvCOSnYphFOg8jCy1e


In [25]:
# Create a headers dictionary with the keys generated

# Set these to values returned above
headers = {"X-API-Key":"LSQHTIXqmeoFxntT31Up", 
           "X-Auth-Token":"TwIJ92oO2PHeWfUe2IyBAhzvCOSnYphFOg8jCy1e"}

In [26]:
# Store credentials for use in other notebooks
with open("credentials.json", "w") as fp:
    json.dump(headers, fp)

# Create organisation

An organisation is a collection of users that share a skills graph and profiles.

In [27]:
# Create a new organisation

new_org = {"name":"Test organization",}

r = requests.post(base_url + "organizations/", json=new_org, headers=headers)
print("Status code", r.status_code)
print("Response", r.json())

organization_id = r.json()["id"]
root_skill_id = r.json()["root_skill"]["id"] # The root node for the skill graph used by this organization


Status code 200
Response {'id': '6ff857ca-09ba-4ab5-a252-c65ecdc2d2c7', 'name': 'Test organization', 'root_skill': {'id': '5dd824ae-3f1c-43be-94f0-a3c5773bf032'}}


In [28]:
# View all organisations

r = requests.get(base_url + "organizations/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'organizations': [{'id': '6ff857ca-09ba-4ab5-a252-c65ecdc2d2c7', 'name': 'Test organization'}]}


In [29]:
# View more detail on one organisation

r = requests.get(base_url + f"organizations/{organization_id}/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'level_map': {'Advanced': 3.0, 'Beginner': 1.0, 'Expert': 4.0, 'Intermediate': 2.0, 'No knowledge': 0.0}, 'name': 'Test organization', 'root_skill': {'id': '5dd824ae-3f1c-43be-94f0-a3c5773bf032'}}


The detail view also includes the skill level map. This can be customized when the organization is initially created.

# Create skills graph

Skills within an organization are structured into a graph.

A sample graph is included in the sample_data directory.

In [30]:
with open("sample_data/graph.json") as fp:
    graph = json.load(fp)

In [31]:
# The graph is stored as nested dictionaries

def add_skills(parent, subgraph):
    for k,v in subgraph.items():
        new_skill = {"name":k, "description":"", "parent":parent}
        r = requests.post(base_url + f"organizations/{organization_id}/skills/", 
                                 json=new_skill, 
                                 headers=headers)
        new_id = r.json()["id"]
        add_skills(new_id, v)

        
add_skills(root_skill_id, graph)

In [32]:
# View skill graph

r = requests.get(base_url + f"organizations/{organization_id}/skills/", headers=headers)
print("Status code", r.status_code)
print(len(r.json()["skills"]))
print("Response", r.json())

def build_full_path(skill):
    if skill["path"] == "Root":
        return skill["name"]
    return skill["path"][5:] + "|" + skill["name"]

all_skills = {build_full_path(i):i for i in r.json()["skills"]}

Status code 200
407
Response {'skills': [{'child_skills': [{'id': '52a0ea59-9d8b-474c-9ee1-ff67340ff873', 'name': 'personal skills'}, {'id': '57c9090f-a988-4892-8ff4-4de11b8f0697', 'name': 'art & design'}, {'id': '39d01059-662f-4d58-b549-135da4cb7048', 'name': 'certifications'}, {'id': '8130f3ee-f699-4710-b853-df9b425cb904', 'name': 'data science'}, {'id': 'f00d564e-fc1b-41e7-ab11-85ddc25cc616', 'name': 'software engineering'}, {'id': 'a73daab0-eb1b-4e22-827a-2746a162fca8', 'name': 'information technology'}, {'id': '9254952e-d9ad-4e7f-ae02-2c3ae0f255d3', 'name': 'engineering'}, {'id': 'be27a8c3-19f3-48f1-b18b-b9f0f51bf9d1', 'name': 'natural sciences'}, {'id': '5155dc27-4c1a-4c90-aeb0-75e30f19a2cc', 'name': 'languages'}, {'id': '7ea1ac4b-5926-400f-ae82-a4f480442141', 'name': 'business & management'}, {'id': '4a77b811-9b25-4ae3-81ae-f4ff33b8be21', 'name': 'social sciences'}, {'id': '41bcea8c-67a0-44b0-9d97-7792eae170b7', 'name': 'mathematics'}, {'id': '3fd14a85-9092-49e7-8ac0-0aa95989e72

In [33]:
len(all_skills)

407

# Create profiles

Profiles are used to collect skills and associate levels.

A variety of sample profiles for specific jobs are available

In [34]:
# Load sample data

profile_dir = 'sample_data/profiles/'

level_map = {'B':'Beginner','I':'Intermediate','A':'Advanced','E':'Expert'}

def skill_id_map(graph, path):
    if len(path) == 1:
        return graph[path[0]]['_meta']['id']
    return skill_id_map(graph[path[0]], path[1:])

profiles = {}

for fn in os.listdir(profile_dir):
    profile_name = fn[:-len('.skills.csv')]
    data = pd.read_csv(os.path.join(profile_dir, fn), header=None)
    profile_competences = {}
    for _,competence in data.iterrows():
        level = competence[0]
        path="|".join([i for i in competence[1:].tolist() if isinstance(i, str)])
        profile_competences[all_skills[path]["id"]] = level_map[level]
    profiles[profile_name] = profile_competences

In [35]:
profiles

{'Applications Engineer': {'166c0836-12b5-4197-b21e-9e11d9c1f315': 'Advanced',
  '27aede2d-6fc8-4194-9f79-8a245f54cf7b': 'Expert',
  '5adb079f-8a48-40aa-a703-ce2916d698af': 'Intermediate',
  '84ddcadc-0f8f-4e78-86d4-244abe74d5da': 'Advanced',
  'b7a33089-c024-4b5c-a820-d4a22ad0db45': 'Advanced',
  'c7183714-9e8c-4a5d-8e4d-56873737b1a7': 'Intermediate',
  'e633e4fd-9d6a-4643-960a-129a62e85c01': 'Advanced'},
 'Business Intelligence Analyst': {'34882f67-2adf-42d3-ab8d-5e5a29c840ab': 'Advanced',
  '509b9d54-6c4b-4c79-988e-b5e3e4500823': 'Expert',
  '52504081-f4ad-46f7-a114-c98ea0910edc': 'Advanced',
  '62b36c29-481a-4e62-9c53-9df4929c1339': 'Advanced',
  'defc37ab-29fe-4cc1-9a75-80926d83019b': 'Advanced'},
 'Chief Information Security Officer': {'09c675bb-d03f-49aa-9dcd-9cac214d34de': 'Expert',
  '12623032-47cd-4353-bc8c-22de86fffbff': 'Advanced',
  '278f8cd4-ad58-4548-86a5-6d4360c4f8d1': 'Expert',
  '3ec872a1-0573-4c9f-9390-ef3318e95d1a': 'Expert',
  '867970c2-604d-4211-a9d6-8d4b23f85729'

In [36]:
# Load all profiles

for profile_name, competences in profiles.items():
    print("Creating profile", profile_name)
    new_profile = {"name":profile_name,
                   "description":"",
                   "type":"job role",
                   "skills":[{"skill":k, "level_name":v} for k,v in competences.items()]
                  }
    r = requests.post(base_url + f"organizations/{organization_id}/profiles/", json=new_profile, headers=headers)
    if r.status_code != 200:
        print("Status code", r.status_code)
        print("Error with", profile_name)
        break

Creating profile Applications Engineer
Creating profile Data Scientist
Creating profile Computer and Information Systems Managers
Creating profile Web Developers
Creating profile Software Engineer
Creating profile Software Engineer ML
Creating profile Systems Engineer
Creating profile DevOps Engineer
Creating profile Machine Learning Researcher
Creating profile Business Intelligence Analyst
Creating profile Software Developers, Applications
Creating profile Cloud architect
Creating profile Sales Engineer
Creating profile Database Administrators
Creating profile Security Engineer
Creating profile Computer Hardware Engineers
Creating profile Data Engineer
Creating profile Product Manager
Creating profile UX Designer
Creating profile Solutions Architect
Creating profile Mobile Developer
Creating profile Machine Learning Engineer
Creating profile Information Security Analysts
Creating profile Data Analyst
Creating profile Full stack developer
Creating profile Systems Administrator
Creating

In [37]:
# List all profiles for an organisation

r = requests.get(base_url + f"organizations/{organization_id}/profiles/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'profiles': [{'description': '', 'id': '800dbb54-25f6-4ac6-8f65-bc8a842f2078', 'name': 'Applications Engineer', 'previous_versions': [], 'skills': [{'level': 4.0, 'level_name': 'Expert', 'skill': '27aede2d-6fc8-4194-9f79-8a245f54cf7b'}, {'level': 2.0, 'level_name': 'Intermediate', 'skill': '5adb079f-8a48-40aa-a703-ce2916d698af'}, {'level': 2.0, 'level_name': 'Intermediate', 'skill': 'c7183714-9e8c-4a5d-8e4d-56873737b1a7'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '166c0836-12b5-4197-b21e-9e11d9c1f315'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': 'e633e4fd-9d6a-4643-960a-129a62e85c01'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '84ddcadc-0f8f-4e78-86d4-244abe74d5da'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': 'b7a33089-c024-4b5c-a820-d4a22ad0db45'}], 'type': 'job role', 'user_id': None}, {'description': '', 'id': '4faf8207-1c65-486b-bc7e-f0c886f0bb11', 'name': 'Data Scientist', 'previous_versions': [], 'skills': [{'level': 4.0, '

# Create users

In [38]:
# Create four users with the competences associated with a job profile

users = [("Adam", profiles["Data Engineer"]), 
         ("Beth", profiles["Electrical Engineer"]), 
         ("Chris", profiles["Mobile Developer"]), 
         ("Dani", profiles["Security Engineer"])]

for user,competences in users:
    print("Creating user", user)
    new_user = {"email":"{0}@test.local".format(user),
                "name":user,
               }
    r = requests.post(base_url + f"organizations/{organization_id}/users/", json=new_user, headers=headers)
    print("Status code", r.status_code)
    print("Response", r.json())
    profile_id = r.json()["profile"]
    
    print("Updating profile")
    update_profile = {"skills":[{"skill":k, "level_name":v} for k,v in competences.items()]}
    r = requests.post(base_url + f"organizations/{organization_id}/profiles/{profile_id}/skills/", 
                      json=update_profile, 
                      headers=headers)
    if r.status_code != 200:
        print("Status code", r.status_code)
        print("Error with", profile_name)
        break
    print(r.json())

Creating user Adam
Status code 200
Response {'email': 'Adam@test.local', 'id': 'b18385f1-7bed-4b98-a8e0-d3b3e7f3f849', 'name': 'Adam', 'profile': 'b856a23e-bdc2-436c-96ec-4d6d9dc486d3'}
Updating profile
{'skills': [{'level': 3.0, 'level_name': 'Advanced', 'skill': '27aede2d-6fc8-4194-9f79-8a245f54cf7b'}, {'level': 4.0, 'level_name': 'Expert', 'skill': '52504081-f4ad-46f7-a114-c98ea0910edc'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '9cb22402-0ea6-4b63-8148-5a81c6ab0093'}, {'level': 4.0, 'level_name': 'Expert', 'skill': '62d3c289-70ce-4467-b2f7-5b79cfcde25f'}, {'level': 4.0, 'level_name': 'Expert', 'skill': 'a0658c74-0678-4784-9e63-cac3a9c19175'}, {'level': 4.0, 'level_name': 'Expert', 'skill': 'aa02f156-4402-48a5-98e7-67aa22647d4f'}, {'level': 4.0, 'level_name': 'Expert', 'skill': '5e25fdba-c00d-4352-95a1-64e5e1a9fdfa'}]}
Creating user Beth
Status code 200
Response {'email': 'Beth@test.local', 'id': '2af63b72-9a9c-4266-9ca5-00fcbfb007c2', 'name': 'Beth', 'profile': '65e035c9-0

In [39]:
# List all users for an organisation

r = requests.get(base_url + f"organizations/{organization_id}/users/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'users': [{'email': 'Adam@test.local', 'id': 'b18385f1-7bed-4b98-a8e0-d3b3e7f3f849', 'name': 'Adam', 'profile': 'b856a23e-bdc2-436c-96ec-4d6d9dc486d3'}, {'email': 'Beth@test.local', 'id': '2af63b72-9a9c-4266-9ca5-00fcbfb007c2', 'name': 'Beth', 'profile': '65e035c9-02ef-4ef8-9c66-29808c1dd565'}, {'email': 'Chris@test.local', 'id': '3945852c-f3bd-45c0-b608-72c449d28387', 'name': 'Chris', 'profile': 'a0ec621d-1acb-4445-8451-29eb7e352ec5'}, {'email': 'Dani@test.local', 'id': '80e6b375-209b-48df-b4b1-55bf16558ea9', 'name': 'Dani', 'profile': 'b35dbf53-8d07-4c6d-8e0e-3deb30556f95'}]}
